In [ ]:
import os 

import numpy as np 

import pandas as pd 

import sagemaker 


sagemaker_session = sagemaker.Session() 

bucket = "<bucket name>" 

prefix = "sagemaker/pytorch-bert-financetext" 

role = sagemaker.get_execution_role() 


inputs_train = sagemaker_session.upload_data("./data/train.csv", bucket=bucket, key_prefix=prefix) 

inputs_test = sagemaker_session.upload_data("./data/test.csv", bucket=bucket, key_prefix=prefix) 

In [ ]:
from sagemaker.pytorch import PyTorch 

output_path = f"s3://{bucket}/{prefix}" 

estimator = PyTorch(
    entry_point="train-dis.py",
    source_dir="code",
    role=role,
    framework_version="1.6",
    py_version="py3",
    instance_count=2,  
    instance_type= "ml.g4dn.12xlarge", 
    output_path=output_path,
    hyperparameters={
        "epochs": 10,
        "lr" : 5e-5,
        "num_labels": 3,
        "train_file": "train.csv",
        "test_file" : "test.csv",
        "MAX_LEN" : 315,
        "batch_size" : 64,
        "test_batch_size" : 10,
        "backend": "nccl"
    },
    
)
estimator.fit({"training": inputs_train, "testing": inputs_test}) 

In [ ]:
from sagemaker.pytorch.model import PyTorchModel 

model_data = estimator.model_data 

pytorch_model = PyTorchModel(model_data=model_data, 

                             role=role, 

                             framework_version="1.6", 

                             source_dir="code", 

                             py_version="py3", 

                             entry_point="inference.py") 

  
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")  

In [ ]:

predictor.serializer = sagemaker.serializers.JSONSerializer() 

predictor.deserializer = sagemaker.deserializers.JSONDeserializer() 

result = predictor.predict("The market is doing better than last year") 

print("predicted class: ", np.argmax(result, axis=1)) 

In [ ]:
predictor.delete_endpoint()